## Assignment 1 Group 3

* Members:
Qianying Diao, Abby (Jingyi) Liu, Xinrong Chen, Qiang Wang

### 1. Read in the datasets

In [1]:
path = 'acquire-valued-shoppers-challenge/offers.csv'
offers_df = spark.read.option('header', 'true').csv(path)
offers_df.printSchema()

root
 |-- offer: string (nullable = true)
 |-- category: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- company: string (nullable = true)
 |-- offervalue: string (nullable = true)
 |-- brand: string (nullable = true)



In [2]:
path = 'acquire-valued-shoppers-challenge/trainHistory.csv'
trainHistory_df = spark.read.option('header', 'true').csv(path)
trainHistory_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- chain: string (nullable = true)
 |-- offer: string (nullable = true)
 |-- market: string (nullable = true)
 |-- repeattrips: string (nullable = true)
 |-- repeater: string (nullable = true)
 |-- offerdate: string (nullable = true)



In [3]:
path = 'acquire-valued-shoppers-challenge/testHistory.csv'
testHistory_df = spark.read.option('header', 'true').csv(path)
testHistory_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- chain: string (nullable = true)
 |-- offer: string (nullable = true)
 |-- market: string (nullable = true)
 |-- offerdate: string (nullable = true)



In [4]:
path = 'acquire-valued-shoppers-challenge/transactions.csv'
transaction_df = spark.read.option('header', 'true').csv(path)
transaction_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- chain: string (nullable = true)
 |-- dept: string (nullable = true)
 |-- category: string (nullable = true)
 |-- company: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- date: string (nullable = true)
 |-- productsize: string (nullable = true)
 |-- productmeasure: string (nullable = true)
 |-- purchasequantity: string (nullable = true)
 |-- purchaseamount: string (nullable = true)



### 3. Drop the unused column.

In [5]:
trainHistory_df = trainHistory_df.drop('repeattrips')
trainHistory_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- chain: string (nullable = true)
 |-- offer: string (nullable = true)
 |-- market: string (nullable = true)
 |-- repeater: string (nullable = true)
 |-- offerdate: string (nullable = true)



In [6]:
trainHistory_df.count()

160057

### 4. Join offer and history dataset

Join dataset 'offers' and dataset 'trainHistory'

In [7]:
joined_train_df = trainHistory_df.join(offers_df, offers_df.offer == trainHistory_df.offer).drop(offers_df.offer)
joined_train_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- chain: string (nullable = true)
 |-- offer: string (nullable = true)
 |-- market: string (nullable = true)
 |-- repeater: string (nullable = true)
 |-- offerdate: string (nullable = true)
 |-- category: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- company: string (nullable = true)
 |-- offervalue: string (nullable = true)
 |-- brand: string (nullable = true)



In [8]:
joined_train_df.count()

160057

Join dataset 'offers' and dataset 'testHistory'

In [9]:
joined_test_df = testHistory_df.join(offers_df, offers_df.offer == testHistory_df.offer).drop(offers_df.offer)
joined_test_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- chain: string (nullable = true)
 |-- offer: string (nullable = true)
 |-- market: string (nullable = true)
 |-- offerdate: string (nullable = true)
 |-- category: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- company: string (nullable = true)
 |-- offervalue: string (nullable = true)
 |-- brand: string (nullable = true)



In [10]:
joined_test_df.count()

151484

### 5. Calculate

To calculate the average purchase quantity and amount, we need to first convert those columns to double

In [11]:
transaction_df = transaction_df.withColumn('purchasequantity', transaction_df['purchasequantity'].cast('double'))
transaction_df = transaction_df.withColumn('purchaseamount', transaction_df['purchaseamount'].cast('double'))
transaction_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- chain: string (nullable = true)
 |-- dept: string (nullable = true)
 |-- category: string (nullable = true)
 |-- company: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- date: string (nullable = true)
 |-- productsize: string (nullable = true)
 |-- productmeasure: string (nullable = true)
 |-- purchasequantity: double (nullable = true)
 |-- purchaseamount: double (nullable = true)



Calculate the average values

In [12]:
transaction = transaction_df.groupby('id','category').avg('purchasequantity','purchaseamount')

In [13]:
joined_train_1 = joined_train_df.join(transaction, (joined_train_df.id == transaction.id) \
                                    & (joined_train_df.category == transaction.category), how = 'left').drop(transaction.id).drop(transaction.category)
joined_train_1.printSchema()

root
 |-- id: string (nullable = true)
 |-- chain: string (nullable = true)
 |-- offer: string (nullable = true)
 |-- market: string (nullable = true)
 |-- repeater: string (nullable = true)
 |-- offerdate: string (nullable = true)
 |-- category: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- company: string (nullable = true)
 |-- offervalue: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- avg(purchasequantity): double (nullable = true)
 |-- avg(purchaseamount): double (nullable = true)



In [14]:
joined_train_1.count()

160057

In [15]:
### For the test dataset:
joined_test_1 = joined_test_df.join(transaction, (joined_test_df.id == transaction.id) \
                                    & (joined_test_df.category == transaction.category), how = 'left').drop(transaction.id).drop(transaction.category)

In [16]:
joined_test_1.printSchema()

root
 |-- id: string (nullable = true)
 |-- chain: string (nullable = true)
 |-- offer: string (nullable = true)
 |-- market: string (nullable = true)
 |-- offerdate: string (nullable = true)
 |-- category: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- company: string (nullable = true)
 |-- offervalue: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- avg(purchasequantity): double (nullable = true)
 |-- avg(purchaseamount): double (nullable = true)



In [17]:
joined_test_1.count()

151484

### 6. Match the records with previous purchase history.

To identify specific item and pair with the customer, we used customer id, the company,category and brand of the item. Join the dataset, if the column "item” matches with id, means this customer had bought this item before, it will return 1, otherwise return 0.

In [17]:
transactions = transaction_df.select("id","company","category","brand").withColumnRenamed('id','item')
transactions = transactions.dropDuplicates()     # To remove multiple purchases

previous = joined_train_1.join(transactions, (joined_train_1.id == transactions.item) & (joined_train_1.company == transactions.company) \
                                    & (joined_train_1.category == transactions.category) & (joined_train_1.brand == transactions.brand)\
                               , how = 'left').drop(transactions.company).drop(transactions.category).drop(transactions.brand)
previous.printSchema()

root
 |-- id: string (nullable = true)
 |-- chain: string (nullable = true)
 |-- offer: string (nullable = true)
 |-- market: string (nullable = true)
 |-- repeater: string (nullable = true)
 |-- offerdate: string (nullable = true)
 |-- category: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- company: string (nullable = true)
 |-- offervalue: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- avg(purchasequantity): double (nullable = true)
 |-- avg(purchaseamount): double (nullable = true)
 |-- item: string (nullable = true)



In [18]:
from pyspark.sql.functions import when

target_train = previous.withColumn("item", when((previous.id==previous.item), 1).otherwise(0))
target_train.printSchema()

root
 |-- id: string (nullable = true)
 |-- chain: string (nullable = true)
 |-- offer: string (nullable = true)
 |-- market: string (nullable = true)
 |-- repeater: string (nullable = true)
 |-- offerdate: string (nullable = true)
 |-- category: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- company: string (nullable = true)
 |-- offervalue: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- avg(purchasequantity): double (nullable = true)
 |-- avg(purchaseamount): double (nullable = true)
 |-- item: integer (nullable = false)



In [19]:
target_train.count()

160057

Same process using on the test Dataset:

In [20]:
previous1 = joined_test_1.join(transactions, (joined_test_1.id == transactions.item) & (joined_test_1.company == transactions.company) \
                                    & (joined_test_1.category == transactions.category) & (joined_test_1.brand == transactions.brand)\
                               , how = 'left').drop(transactions.company).drop(transactions.category).drop(transactions.brand)

In [21]:
target_test = previous1.withColumn("item", when((previous1.id==previous1.item), 1).otherwise(0))
target_test.printSchema()

root
 |-- id: string (nullable = true)
 |-- chain: string (nullable = true)
 |-- offer: string (nullable = true)
 |-- market: string (nullable = true)
 |-- offerdate: string (nullable = true)
 |-- category: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- company: string (nullable = true)
 |-- offervalue: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- avg(purchasequantity): double (nullable = true)
 |-- avg(purchaseamount): double (nullable = true)
 |-- item: integer (nullable = false)



In [23]:
target_test.count()

151484

### 7. Subset and export as csv file.

In [35]:
subset_train = target_train.filter((target_train.id < 107014607) & (target_train['avg(purchasequantity)']>0)\
                                   & (target_train['avg(purchaseamount)']>0))

In [36]:
subset_train.toPandas().to_csv('result.csv')